In [1]:
import numpy as np
import h5py
from glob import glob
from keras.models import load_model

diseases = ['Atelectasis']
def find_last_checkpoint(disease):
    last_model_point =0
    for f in (glob('generated_models/Generator_model_{}_*'.format(disease))):
        file = f.split('/')[-1]
        checkpoint_no = int(file.split('_')[-1])
        if checkpoint_no > last_model_point:
            last_model_point = checkpoint_no
    
    return int(last_model_point)

generator_paths = []
for disease in diseases:
    generator_path = 'generated_models/Generator_model_{}_{}'.format(disease,find_last_checkpoint(disease))
    generator_paths.append(generator_path)
    
print (generator_paths)


c:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['generated_models/Generator_model_Atelectasis_4000']


In [2]:
data = h5py.File('chest_xray.h5','a') 
        
print (data['images'].shape)


(20000, 128, 128, 1)


In [19]:
disease_vec_labels = ['Atelectasis','Cardiomegaly','Consolidation','Edema','Effusion','Emphysema','Fibrosis',
 'Hernia','Infiltration','Mass','Nodule','Pleural_Thickening','Pneumonia','Pneumothorax']
number_of_images = 200
final_data = {}

for generator_path in generator_paths:
    generator = load_model(generator_path)
    noise = np.random.normal(0,1,(number_of_images,100))
    generated_img = generator.predict(noise)
    generated_img = 0.5* generated_img + 0.5
    print (generated_img.shape)
    
    final_data['images']=np.array(data.get('images'))
    final_data['images']=np.append(final_data['images'],generated_img,axis=0)
    print (final_data['images'].shape)
    disease_generator = (generator_path.split('/')[-1]).split('_')[-2]
    print (disease_generator)
    
    for disease in disease_vec_labels:
        if disease == disease_generator:
            final_data[disease] = np.append(np.array(data.get(disease)), [1] * number_of_images)
        else:
            final_data[disease] = np.append(np.array(data.get(disease)), [0] * number_of_images)
    


c:\python35\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(200, 128, 128, 1)
(20200, 128, 128, 1)
Atelectasis


In [20]:
for key in final_data.keys():
    print (key,final_data[key].shape)

Pleural_Thickening (20200,)
Edema (20200,)
Cardiomegaly (20200,)
Infiltration (20200,)
images (20200, 128, 128, 1)
Consolidation (20200,)
Mass (20200,)
Effusion (20200,)
Pneumothorax (20200,)
Fibrosis (20200,)
Pneumonia (20200,)
Hernia (20200,)
Nodule (20200,)
Emphysema (20200,)
Atelectasis (20200,)


In [34]:
data.close()
from tqdm import tqdm

def write_dict_as_hdf(out_path,out_dict,compression='gzip'):
    with h5py.File(out_path,'w') as h:
        for k, v in out_dict.items():
            s_data = np.stack(v,0)
            h.create_dataset(k, data = s_data,compression = compression)
                
                    
               
            

In [35]:
write_dict_as_hdf('final_xray.h5',final_data,compression=None)

In [36]:
with h5py.File('final_xray.h5','r') as h_data:
    for key in h_data.keys():
        print (key,h_data[key].shape)

Atelectasis (20200,)
Cardiomegaly (20200,)
Consolidation (20200,)
Edema (20200,)
Effusion (20200,)
Emphysema (20200,)
Fibrosis (20200,)
Hernia (20200,)
Infiltration (20200,)
Mass (20200,)
Nodule (20200,)
Pleural_Thickening (20200,)
Pneumonia (20200,)
Pneumothorax (20200,)
images (20200, 128, 128, 1)
